In [ ]:
#r "./../../../../../../public/src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
using System;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;
using BoSSS.Application.XNSE_Solver;

using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [ ]:
// set parameterz
int[] core_sweep = {64};
int[] PolyDegS = new int[] {3};
int[] ResArray = new int[] {15};
int MemoryPerCore = 2000;

In [ ]:
bool useAMR = true;
bool useLoadBal = true;
int NoOfTimeSteps = 60;
bool Steady = false;
bool IncludeConvection = true;
var Gshape = Shape.Cube;

In [ ]:
ExecutionQueues

index,type,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths,Username,Password,ServerName,ComputeNodes,DefaultJobPriority,SingleNode,PrivateKeyFilePath,AdditionalBatchCommands,DeploymentBaseDirectoryAtRemote,SlurmAccount,Email,MonoDebug
0,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,C:\Users\weber\AppData\Local\BoSSS-LocalJobs,False,LocalPC,dotnet,<null>,,,,,,,,,,,,,
1,BoSSS.Application.BoSSSpad.MsHPC2012Client,\\hpccluster\hpccluster-scratch\weber\dply,True,FDY-cluster,dotnet,,"[ { \\hpccluster\hpccluster-scratch\weber == : LocalMountPath: \\hpccluster\hpccluster-scratch\weber, PathAtRemote: } ]",FDY\weber,<null>,DC2,[ HPCCLUSTER2 ],Normal,True,,,,,,
2,BoSSS.Application.BoSSSpad.SlurmClient,X:\Deployerie,True,<null>,dotnet,,,jw52xeqa,<null>,lcluster5.hrz.tu-darmstadt.de,,,,C:\Users\weber\.ssh\id_rsa,<null>,/home/jw52xeqa/Deployerie,project01217,<null>,False
3,BoSSS.Application.BoSSSpad.SlurmClient,X:\Deployerie,True,HHLR,dotnet,,"[ { W:\work\scratch\jw52xeqa == /work/scratch/jw52xeqa: LocalMountPath: W:\work\scratch\jw52xeqa, PathAtRemote: /work/scratch/jw52xeqa } ]",jw52xeqa,<null>,lcluster14.hrz.tu-darmstadt.de,,,,C:\Users\weber\.ssh\id_rsa,"[ #SBATCH -p test24, #SBATCH -C avx512 ]",/home/jw52xeqa/Deployerie,special00006,<null>,False


In [ ]:
using BoSSS.Application.BoSSSpad;

In [ ]:
// ==================================
// setup Client & Workflow & Database
// ==================================
var myBatch = (SlurmClient)ExecutionQueues[3];
//new string[]{"#SLURM -p test24", "#SLURM -C avx512","#SLURM -N 3"};
var AddSbatchCmds = new List<string>();
//AddSbatchCmds.AddRange(myBatch.AdditionalBatchCommands);
//AddSbatchCmds.AddRange(new string[]{"#SBATCH --mem-per-cpu="+MemoryPerCore});
AddSbatchCmds.AddRange(new string[]{"#SBATCH -N 4","#SBATCH -p test24", "#SBATCH -C avx512", "#SBATCH --mem-per-cpu="+MemoryPerCore});
myBatch.AdditionalBatchCommands = AddSbatchCmds.ToArray();
myBatch.AdditionalBatchCommands

index,value
0,#SBATCH -N 4
1,#SBATCH -p test24
2,#SBATCH -C avx512
3,#SBATCH --mem-per-cpu=2000


In [ ]:
//var myBatch = (MsHPC2012Client)ExecutionQueues[1];

In [ ]:
string WFlowName = "rotCube+Inlet_showcase";
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.Init(WFlowName);
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();

Project name is set to 'rotCube+Inlet_showcase'.


In [ ]:
myBatch.AllowedDatabasesPaths.Clear();
//string dirname = "DB_rotCubePaper";
//string dirname = "DB_rotSphere_CoreScaling";
//string dirname = "DB_rotSphere_DOFScaling";
string dirname ="DB_rotSphereBenchmark";
//string winpath = @"W:\work\scratch\jw52xeqa\"+dirname;
//string remotepath = @"/work/scratch/jw52xeqa/"+dirname;
string winpath = @"\\hpccluster\hpccluster-scratch\"+dirname;
var myDB = OpenOrCreateDatabase(winpath);
//myBatch.AllowedDatabasesPaths.Add(new AllowedDatabasesPair(winpath,remotepath)); myBatch.AllowedDatabasesPaths

Opening existing database '\\hpccluster\hpccluster-scratch\DB_rotSphereBenchmark'.


In [ ]:
static class Utils {
    // DOF per cell in 3D
    public static int Np(int p) {
        return (p*p*p + 6*p*p + 11*p + 6)/6;
    }    
}

In [ ]:
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;

using 

In [ ]:
double xMax = 3.0, yMax = 1.0, zMax = 1.0;
double xMin = -3.0, yMin = -1.0,zMin = -1.0;

double xMax = 1.0, yMax = 1.0, zMax=1.0;
double xMin = -1.0, yMin = -1.0,zMin = -1.0;

In [ ]:
List<IGridInfo> Grids = new List<IGridInfo>();
foreach(var Res in ResArray){
    var _xNodes = GenericBlas.Linspace(xMin, xMax, 2*Res + 1);
    var _yNodes = GenericBlas.Linspace(yMin, yMax, Res + 1);
    var _zNodes = GenericBlas.Linspace(zMin, zMax, Res + 1);

    GridCommons grd;
    string gname = "RotBenchmarkGrid";
    
    var tmp = new List<IGridInfo>();
    foreach(var grid in myDB.Grids){
        try{
            bool IsMatch = grid.Name.Equals(gname)&&grid.NumberOfCells==Res*Res*Res;
            if(IsMatch) tmp.Add(grid);
        }
        catch(Exception ex) {
            Console.WriteLine(ex.Message);
        }
    }
    //var tmp = myDB.Grids.Where(g=>g.Name.Equals(gname)&&g.NumberOfCells==Res*Res*Res); // this leads to exception in case of broken grids
    /*
    if(tmp.Count()>=1){
        Console.WriteLine("Grid found: "+tmp.Pick(0).Name);
        Grids.Add(tmp.Pick(0));
        continue;
    }
    */
    
    grd = Grid3D.Cartesian3DGrid(_xNodes, _yNodes, _zNodes);
    grd.Name = gname;
    //grd.AddPredefinedPartitioning("debug", MakeDebugPart);

    grd.EdgeTagNames.Add(1, "Velocity_inlet");
    grd.EdgeTagNames.Add(2, "Wall");
    grd.EdgeTagNames.Add(3, "Pressure_Outlet");

    grd.DefineEdgeTags(delegate (double[] _X) {
        var X = _X;
        double x, y, z;
        x = X[0];
        y = X[1];
        z = X[2];
        if(Math.Abs(x-xMin)<1E-8)
            return 1;
        else
            return 3;
    });
    myDB.SaveGrid(ref grd,true);
    Grids.Add(grd);
} Grids

Object reference not set to an instance of an object.
Object reference not set to an instance of an object.
Object reference not set to an instance of an object.
Grid Edge Tags changed.


#0: { Guid = 220a0aa4-950c-4c5f-9dbb-b5df8fc38522; Name = RotBenchmarkGrid; Cell Count = 6750; Dim = 3 }


In [ ]:
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Control;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.NSECommon;
using BoSSS.Application.XNSE_Solver.LoadBalancing;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.XdgTimestepping;

In [ ]:
int SpaceDim = 3;

In [ ]:
Func<IGridInfo, int, double, XNSE_Control> GenXNSECtrl = delegate(IGridInfo grd, int k, double Viscosity){
    XNSE_Control C = new XNSE_Control();
    // basic database options
    // ======================
    C.AlternateDbPaths = new[] {
        (@"/work/scratch/jw52xeqa/DB_rotSphereBenchmark", ""),
        (@"W:\work\scratch\jw52xeqa\DB_rotSphereBenchmark","")};
    //C.DbPath=@"\\hpccluster\hpccluster-scratch\DB_rotSphereBenchmark";
    C.savetodb = true;
    int J  = grd.NumberOfCells;
    C.SessionName = string.Format("J{0}_k{1}_t{2}", J, k,NoOfTimeSteps);
    if(IncludeConvection){
        C.SessionName += "_NSE";
        C.Tags.Add("NSE");
    } else {
        C.SessionName += "_Stokes";
        C.Tags.Add("Stokes");
    }
    C.Tags.Add(SpaceDim + "D");
    if(Steady)C.Tags.Add("steady");
    else C.Tags.Add("transient");

    // DG degrees
    // ==========
    C.SetFieldOptions(k, Math.Max(k, 2));
    C.saveperiod = 1;
    C.TracingNamespaces = "*";

    C.GridGuid = grd.ID;
    C.GridPartType = GridPartType.clusterHilbert;
    C.DynamicLoadbalancing_ClassifierType = ClassifierType.CutCells;
    C.DynamicLoadBalancing_On = useLoadBal;
    C.DynamicLoadBalancing_RedistributeAtStartup = true;
    C.DynamicLoadBalancing_Period = 1;
    C.DynamicLoadBalancing_ImbalanceThreshold = 0.1;


    // Physical Parameters
    // ===================
    const double rhoA = 1;
    const double Re = 100;
    double muA = Viscosity;
    
    double partRad = 0.4111;
    double anglev = Re*muA/rhoA/(2*partRad);
    double zSpan = Math.Abs(zMax-zMin);
    double ySpan = Math.Abs(yMax-yMin);
    double d_hyd = zSpan*ySpan/(2*zSpan+2*ySpan);
    double VelocityIn = Re*muA/rhoA/d_hyd;
    double[] pos = new double[SpaceDim];

    C.PhysicalParameters.IncludeConvection = IncludeConvection;
    C.PhysicalParameters.Material = true;
    C.PhysicalParameters.rho_A = rhoA;
    C.PhysicalParameters.mu_A = muA;

    C.Rigidbody.SetParameters(pos,anglev,partRad,SpaceDim);
    C.Rigidbody.SpecifyShape(Gshape);
    C.Rigidbody.SetRotationAxis("x");
    C.AddInitialValue(VariableNames.LevelSetCGidx(0), new Formula("X => -1"));
    C.UseImmersedBoundary = true;
    
    C.AddInitialValue("Pressure", new Formula(@"X => 0"));
    C.AddBoundaryValue("Pressure_Outlet");
    C.AddBoundaryValue("Velocity_inlet","VelocityX",new Formula($"(X) => {VelocityIn}"));

    C.CutCellQuadratureType = BoSSS.Foundation.XDG.XQuadFactoryHelper.MomentFittingVariants.Saye;
    C.UseSchurBlockPrec = true;
    C.AgglomerationThreshold = 0.1;
    C.AdvancedDiscretizationOptions.ViscosityMode = ViscosityMode.FullySymmetric;
    C.Option_LevelSetEvolution2 = LevelSetEvolution.Prescribed;
    C.Option_LevelSetEvolution = LevelSetEvolution.None;
    C.Timestepper_LevelSetHandling = LevelSetHandling.None;
    C.LinearSolver.NoOfMultigridLevels = 4;
    C.LinearSolver.ConvergenceCriterion = 1E-8;
    C.LinearSolver.MaxSolverIterations = 200;
    C.LinearSolver.MaxKrylovDim = 50;
    C.LinearSolver.TargetBlockSize = 10000;
    C.LinearSolver.verbose = true;
    C.LinearSolver.SolverCode = LinearSolverCode.exp_Kcycle_schwarz;
    C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    C.NonLinearSolver.ConvergenceCriterion = 1E-6;
    C.NonLinearSolver.MaxSolverIterations = 10;
    C.NonLinearSolver.verbose = true;

    C.AdaptiveMeshRefinement = useAMR;
    if (useAMR) {
        C.SetMaximalRefinementLevel(2);
        C.AMR_startUpSweeps = 0;
    }

    // Timestepping
    // ============
    double dt = -1;
    if(Steady){
        C.TimesteppingMode = AppControl._TimesteppingMode.Steady;
        dt = 1000;
        C.NoOfTimesteps = 1;
    } else {
        C.TimesteppingMode = AppControl._TimesteppingMode.Transient;        
        dt = 0.05;        
        C.NoOfTimesteps = NoOfTimeSteps;
    }
    C.TimeSteppingScheme = TimeSteppingScheme.BDF2;
    C.dtFixed = dt;
    return C;
};

In [ ]:
var ViscositySweep = new double[]{1,1E-1,1E-2,1E-3};
List<XNSE_Control> controls = new List<XNSE_Control>();
foreach(IGridInfo grd in Grids){
    foreach(int k in PolyDegS){
        foreach(double v in ViscositySweep)
            controls.Add(GenXNSECtrl(grd,k,v));
    }
} controls.Select(s=>s.SessionName)

index,value
0,J6750_k3_t60_NSE


In [ ]:
// ==========
// Launch Job
// ==========
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.BoSSSpad;

In [ ]:
controls.Select(s=>s.SessionName)

index,value
0,J6750_k3_t60_NSE


In [ ]:
foreach(var ctrl in controls){
    string sessname = ctrl.SessionName;
    foreach(int cores in core_sweep){
        ctrl.SessionName = sessname + "_c"+cores+"_mue_"+ctrl.PhysicalParameters.mu_A;
        var aJob   = new Job("rotting_"+Gshape+ctrl.SessionName,typeof(XNSE));
        aJob.SetControlObject(ctrl);
        aJob.NumberOfMPIProcs         = cores;
        aJob.ExecutionTime            = "1:00:00";
        aJob.UseComputeNodesExclusive = true;
        aJob.Activate(myBatch);
    }
}

Deploying job rotting_CubeJ6750_k3_t60_NSE_c64_c64 ... 
Grid 220a0aa4-950c-4c5f-9dbb-b5df8fc38522 is not present in database - copy to target system...
done.
Deploying executables and additional files ...
Deployment directory: X:\Deployerie\rotCube+Inlet_showcase-XNSE_Solver2021Sep11_091726
copied 49 files.
   written file: control.obj
   copied 'amd64' runtime.
deployment finished.
21407919

